# Delivery time prediction

Justyna Gibas 403122 <br>
Jan Pomykacz

## Introduction
The goal of our project is to predict food delivery time. Model can be used by companies to estimate delivery time based on distance, order time etc, and notify their customers when food should arrive.